In [13]:
import keras, os
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from PIL import Image, ImageFile

In [14]:
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [15]:
train_datagen = ImageDataGenerator(
    rescale=1./255, shear_range=0.1, zoom_range=0.2, horizontal_flip=False
)
test_datagen = ImageDataGenerator(rescale=1./255)

In [16]:
train_set = train_datagen.flow_from_directory(
    "Data\Train", 
    target_size=(300, 300), batch_size=16, class_mode='binary'
)
test_set = test_datagen.flow_from_directory(
    "Data\Test", 
    target_size=(300, 300), batch_size=16, class_mode='binary'
)

Found 2004 images belonging to 2 classes.
Found 1800 images belonging to 2 classes.


<>:2: SyntaxWarning: invalid escape sequence '\T'
<>:6: SyntaxWarning: invalid escape sequence '\T'
<>:2: SyntaxWarning: invalid escape sequence '\T'
<>:6: SyntaxWarning: invalid escape sequence '\T'
C:\Users\Dell\AppData\Local\Temp\ipykernel_24368\1359225472.py:2: SyntaxWarning: invalid escape sequence '\T'
  "Data\Train",
C:\Users\Dell\AppData\Local\Temp\ipykernel_24368\1359225472.py:6: SyntaxWarning: invalid escape sequence '\T'
  "Data\Test",


In [17]:
model = Sequential()

# Input and convolutional layers
model.add(Conv2D(input_shape=(300, 300, 3), filters=16, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPool2D(pool_size=(2, 2)))

# Global Average Pooling instead of Flatten
model.add(GlobalAveragePooling2D())

# Dense layers
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # Binary classification

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

c:\Users\Dell\OneDrive\Desktop\BTP Live Detectness\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_13 (Conv2D)              │ (None, 300, 300, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 150, 150, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 150, 150, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 75, 75, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 75, 75, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 37, 37, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 64)             │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 32,033 (125.13 KB)

 Trainable params: 32,033 (125.13 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [22]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Update the checkpoint filepath
checkpoint = ModelCheckpoint("vgg16_1.keras", monitor='val_accuracy', verbose=1, save_best_only=True)
early = EarlyStopping(monitor='val_accuracy', patience=20, verbose=1)

# Correct the fit method
model.fit(
    train_set,  # Use the generator directly
    validation_data=test_set,
    steps_per_epoch=100, 
    epochs=100, 
    validation_steps=10, 
    callbacks=[checkpoint, early]
)

c:\Users\Dell\OneDrive\Desktop\BTP Live Detectness\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 288ms/step - accuracy: 0.7455 - loss: 0.5805

c:\Users\Dell\OneDrive\Desktop\BTP Live Detectness\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()



Epoch 1: val_accuracy improved from -inf to 0.86250, saving model to vgg16_1.keras
100/100 ━━━━━━━━━━━━━━━━━━━━ 32s 301ms/step - accuracy: 0.7456 - loss: 0.5803 - val_accuracy: 0.8625 - val_loss: 0.3906
Epoch 2/100
 26/100 ━━━━━━━━━━━━━━━━━━━━ 21s 284ms/step - accuracy: 0.7473 - loss: 0.5707

E:\ANACONDA\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)



Epoch 2: val_accuracy did not improve from 0.86250
100/100 ━━━━━━━━━━━━━━━━━━━━ 8s 80ms/step - accuracy: 0.7457 - loss: 0.5613 - val_accuracy: 0.8062 - val_loss: 0.5346
Epoch 3/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 284ms/step - accuracy: 0.7474 - loss: 0.5373
Epoch 3: val_accuracy did not improve from 0.86250
100/100 ━━━━━━━━━━━━━━━━━━━━ 29s 292ms/step - accuracy: 0.7473 - loss: 0.5373 - val_accuracy: 0.7625 - val_loss: 0.4488
Epoch 4/100
 26/100 ━━━━━━━━━━━━━━━━━━━━ 20s 275ms/step - accuracy: 0.7639 - loss: 0.5022
Epoch 4: val_accuracy did not improve from 0.86250
100/100 ━━━━━━━━━━━━━━━━━━━━ 7s 74ms/step - accuracy: 0.7554 - loss: 0.5129 - val_accuracy: 0.7500 - val_loss: 0.4150
Epoch 5/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 278ms/step - accuracy: 0.7229 - loss: 0.4941
Epoch 5: val_accuracy did not improve from 0.86250
100/100 ━━━━━━━━━━━━━━━━━━━━ 28s 283ms/step - accuracy: 0.7230 - loss: 0.4939 - val_accuracy: 0.8375 - val_loss: 0.5205
Epoch 6/100
 26/100 ━━━━━━━━━━━━━━━━━━━━ 20s 281ms/

In [23]:
# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(
    test_set,  # Test data generator or dataset
    steps=10  # Number of batches to evaluate
)

# Print test results
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 82ms/step - accuracy: 0.9139 - loss: 0.2873
Test Loss: 0.2873830199241638
Test Accuracy: 0.8999999761581421
